---
jupyter: python3
toc: true
date: 2022-05-26
title: Jupyter Lab Server 세팅
categories: [JupyterLab]
author: limyj0708
comments:
  giscus: 
    repo: quarto-dev/quarto-docs
---

***
- CentOS에 Jupyter Lab 설치가 완료되었다고 가정하자.
  - 구글에 Jupyter lab server라고 검색하면, 아래 페이지가 가장 먼저 뜨게 되는데, 이 페이지 말고
    - https://jupyter-notebook.readthedocs.io/en/stable/public_server.html
  - 이 페이지를 확인하는 것이 좋다.
    - https://jupyter-server.readthedocs.io/en/latest/operators/public-server.html

### 1. Jupyter server configuration 파일 설정
1. `jupyter server --generate-config`를 하면, /home/"유저이름"/.jupyter/jupyter_server_config.py가 생성된다. 여기에서 세팅을 해야 한다.
   - c.ServerApp.open_browser = False (브라우저 띄우지 않음)
   - c.ServerApp.password = 'argon2...'
     - `from jupyter_server.auth import passwd; passwd()`를 실행하여 생성하는, 암호화된 비밀번호를 입력한다. 
   - c.ServerApp.port = 원하는 포트
   - c.ServerApp.certfile = openssl로 만든 certfile 등록 (예: mycert.pem)
   - c.ServerApp.keyfile = openssl로 만든 keyfile 등록 (예: mykey.key)
   - c.ServerApp.ip = '*', 혹은 접근 가능하게 하고싶은 ip
   - c.ServerApp.root_dir = 원하는 경로
     - notebook_dir is deprecated, use root_dir
   - c.ServerApp.allow_origin = '*'
     -  Use '*' to allow any origin to access your server.

### 2. 문제점과 해결
1. self-signed certificate 오류 메세지 
   1. `openssl req -x509 -nodes -days 999 -newkey rsa:2048 -keyout mykey.key -out mycert.pem`
      - 이런 식으로 만든 self-signed certificate를 쓰면 jupyter lab 실행 시, 뭘 하기만 하면 SSL Error를 띄운다.
        - 이런 식으로 : SSL Error on 13 ('ip', 13786): [SSL: SSLV3_ALERT_CERTIFICATE_UNKNOWN] sslv3 alert certificate unknown (_ssl.c:997)
        - Safari에서는 안 뜨고, Edge에서는 뜨는 걸로 봐서 크로미움 기반 브라우저에서 접속하면 뜨는 것 같다.
    2. Let's Encrypt 같은 서비스를 이용해서 인증서를 받아도 되는데, 도메인 네임도 없는, 혼자 쓰는 무료 클라우드 서버에서 그렇게까지 해야 하나 싶다.
    3. tmux에서 새 pane을 만들고, `jupyter lab > /dev/null 2>&1 &`으로 jupyter lab을 실행하여 콘솔 output을 없애고 백그라운드에서 jupyter lab을 실행하자.